In [26]:
import pandas as pd
import numpy as np

In [27]:
d = pd.read_csv("./hists_imgs.csv")

In [28]:
d.head()

,0,1,2,3,4,tag,local_path
0,0.000000,0.000000,0.000000,0.029815,1.000000,person,/Users/damoncrockett/Desktop/stpete/viz/hists/...
1,0.000000,0.000000,0.000000,0.000000,1.000000,hair,/Users/damoncrockett/Desktop/stpete/viz/hists/...
2,0.000000,0.000000,0.156982,1.000000,0.412402,hairstyle,/Users/damoncrockett/Desktop/stpete/viz/hists/...
3,1.000000,0.766667,0.444203,0.062319,0.000000,selfie,/Users/damoncrockett/Desktop/stpete/viz/hists/...
4,0.105419,0.544828,0.564532,0.741872,1.000000,vehicle,/Users/damoncrockett/Desktop/stpete/viz/hists/...


In [29]:
d = d[d.local_path.notnull()]
d.reset_index(drop=True,inplace=True)

In [30]:
from sklearn.manifold import TSNE as tsne
from PIL import Image, ImageDraw

In [31]:
thumb_side = 128

## Fill

In [32]:
from shapely.geometry import Point

In [33]:
crit = np.sqrt(2) # adjacency

In [34]:
PRE = "/Users/damoncrockett/Desktop/stpete/viz/hists-tsne/"

In [37]:
num_bin = 3 * int(round( np.sqrt( len(d) / np.pi ) ))

X = d.iloc[:,:5].as_matrix()
tsne_obj = tsne(n_components=2)

subspace_tsne = pd.DataFrame(tsne_obj.fit_transform(X),columns=["x","y"])
subspace_tsne['grid_x'] = pd.cut(subspace_tsne['x'],num_bin,labels=False)
subspace_tsne['grid_y'] = pd.cut(subspace_tsne['y'],num_bin,labels=False)
subspace_tsne['local_path'] = d.local_path
subspace_tsne['tag'] = d.tag

grid_side = num_bin 
px_w = thumb_side * grid_side
px_h = thumb_side * grid_side

x,y = range(grid_side) * grid_side, np.repeat(range(grid_side),grid_side)
grid_list = pd.DataFrame(x,columns=['x'])
grid_list['y'] = y

point = []
for i in range(len(grid_list)):
    point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))

grid_list['point'] = point

avg_dist = []
open_grid = list(grid_list.point)
canvas = Image.new('RGB',(px_w,px_h),(255,255,255))

plotted = 0
for i in np.random.permutation(np.arange(np.round(len(subspace_tsne)))):
    im = Image.open(subspace_tsne.local_path.loc[i])
    tag = subspace_tsne.tag.loc[i]
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    
    draw = ImageDraw.Draw(im)
    draw.text((5,110),tag,fill=(155,155,155))
    
    x = subspace_tsne.grid_x.loc[i] * thumb_side 
    y = subspace_tsne.grid_y.loc[i] * thumb_side 
    grid_point = Point((subspace_tsne.grid_x.loc[i],subspace_tsne.grid_y.loc[i]))

    if grid_point in open_grid:
        canvas.paste(im,(x,y))
        open_grid.remove(grid_point)
        plotted+=1

    elif len(open_grid) == 0:
        break

    else:
        closest_open = min(open_grid,key=lambda x: grid_point.distance(x))
        if grid_point.distance(closest_open) < crit: # adjacency criterion
            x = int(closest_open.x) * thumb_side
            y = int(closest_open.y) * thumb_side
            canvas.paste(im,(x,y))
            open_grid.remove(closest_open)
            avg_dist.append( float(grid_point.distance(closest_open)) )
            plotted+=1

proportion = str(int(round(plotted*100/len(subspace_tsne))))
canvas.save(PRE+str(num_bin)+"_"+proportion+"_"+"_"+str(plotted)+".png")